# 🕵️‍♂️ Knowledge Graph Generator from Audit Trails
This notebook extracts triples from audit logs or event text and builds a knowledge graph using RDFLib and NetworkX.
- Input: CSV with event/action logs
- Output: RDF Graph + visualization
- Tools: RDFLib, NetworkX, Pandas, Gradio

In [ ]:
# ✅ Install dependencies
!pip install rdflib networkx pandas matplotlib gradio

In [ ]:
# 📦 Imports
import pandas as pd
import rdflib
import networkx as nx
import matplotlib.pyplot as plt
from rdflib import Graph, URIRef, Namespace
import gradio as gr
import io

In [ ]:
# 🧠 Convert CSV events to RDF knowledge graph
def build_kg(file):
    df = pd.read_csv(file.name)
    required_cols = {'Actor', 'Action', 'Target'}
    if not required_cols.issubset(set(df.columns)):
        return "CSV must contain: Actor, Action, Target columns"

    g = Graph()
    NS = Namespace("http://example.org/audit/")

    for _, row in df.iterrows():
        s = URIRef(NS[row['Actor'].replace(" ", "_")])
        p = URIRef(NS[row['Action'].replace(" ", "_")])
        o = URIRef(NS[row['Target'].replace(" ", "_")])
        g.add((s, p, o))

    # Visualize with NetworkX
    nxg = nx.DiGraph()
    for s, p, o in g:
        nxg.add_edge(s.split('/')[-1], o.split('/')[-1], label=p.split('/')[-1])

    pos = nx.spring_layout(nxg)
    plt.figure(figsize=(10, 6))
    nx.draw(nxg, pos, with_labels=True, node_color='skyblue', node_size=1500, font_size=10)
    edge_labels = nx.get_edge_attributes(nxg, 'label')
    nx.draw_networkx_edge_labels(nxg, pos, edge_labels=edge_labels)
    plt.title("Audit Trail Knowledge Graph")
    plt.savefig("kg_audit.png")
    return "kg_audit.png"

In [ ]:
# 🎛️ Gradio UI
gr.Interface(
    fn=build_kg,
    inputs=gr.File(label="Upload Audit Trail CSV (Actor, Action, Target)"),
    outputs=gr.Image(label="Knowledge Graph Visualization"),
    title="🕵️‍♂️ Audit Trail Knowledge Graph",
    description="Upload event/action logs and visualize interactions as a knowledge graph"
).launch()